In [8]:
import requests
from requests import get
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from fake_useragent import UserAgent
from lxml import etree
from time import sleep
from random import randint
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pymysql
import random
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import time
import urllib.request

In [9]:
# Selenium class에서 성분 추출
def execute_element(selenium_class):
    global results
    for element in selenium_class:
        results = element.text
    return results

In [12]:
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
    #options.add_argument("headless")
    #options.add_argument('--start-fullscreen')
driver = webdriver.Chrome(options = options)
driver.set_window_size(1920,500)

totalWidth = driver.execute_script("return document.body.offsetWidth")
totalHeight = driver.execute_script('return document.body.parentNode.scrollHeight')
# 화면의 실제 사이즈로 변경합니다.
driver.set_window_size(totalWidth, totalHeight)

In [18]:
def movie_crawl(movie_name,driver):
    options = webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["enable-logging"])
    #options.add_argument("headless")
    #options.add_argument('--start-fullscreen')
    driver = webdriver.Chrome(options = options)
    driver.get(f'https://www.imdb.com/search/')
    driver.set_window_size(1920,500)

    totalWidth = driver.execute_script("return document.body.offsetWidth")
    totalHeight = driver.execute_script('return document.body.parentNode.scrollHeight')
    # 화면의 실제 사이즈로 변경합니다.
    driver.set_window_size(totalWidth, totalHeight)
    # 네이버 웹툰 페이지를 불러와서 웹툰 제목에 해당하는 결과물 출력
    
    time.sleep(2)
      
    try:
        search_box = driver.find_element(By.XPATH, '//*[@id="suggestion-search"]')
        search_box.send_keys(movie_name)
        time.sleep(3)
        driver.find_element(By.XPATH, '//*[@id="react-autowhatever-1--item-0"]/a').click()
        time.sleep(2)
        title = execute_element(driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/h1'))
         # search_box.send_keys(Keys.RETURN)
         # 각 해당정보 추출
        rating = execute_element(driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[2]/div/div[1]/a/div/div/div[2]/div[1]/span[1]')) 
        genre1 = execute_element(driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/div[1]/div/div[2]/a[1]/span'))
        genre2 = execute_element(driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/div[1]/div/div[2]/a[2]/span'))
        story = execute_element(driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/div[1]/p/span[3]'))
        year = execute_element(driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/div/ul/li[1]/a'))
        movie_rate = execute_element(driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/div/ul/li[2]/a'))
        director = execute_element(driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/div[3]/ul/li[1]/div/ul/li/a'))
        lead_actor = execute_element(driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/div[3]/ul/li[3]/div/ul/li[1]'))
        sup_actor = execute_element(driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/div[3]/ul/li[3]/div/ul/li[2]'))
        score = execute_element(driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[2]/ul/li[3]/a/span/span[1]/span'))
        vote = execute_element(driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[2]/div/div[1]/a/div/div/div[2]/div[3]'))
        time.sleep(2)
        driver.find_element(By.XPATH,'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[1]/div/div[1]/div/a/div').click()
        time.sleep(2)
        image = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/div[4]/img').get_attribute("src")
        # urllib.request.urlretrieve(image, "../image/str(movie_name)"+".jpg")
        image = image.replace("'",'')    
    except:
        driver.close()
        return None
    
    time.sleep(3)
    
   
    def convert_unit(value):
        unit = value[-1:] # 값의 맨마지막 값을 저장
        value = re.sub(r'[KM]', '',value) # K,M 이 있으면 제거
        if unit == 'K':                     # sales의 마지막 값에 K가 있으면 천을 곱하고 이외의 경우에는 100000 을 곱해준다
            value = float(value) * 1000
            return value
        elif unit =='M':
            value = float(value) * 100000
            return value
        else:
            return value
    vote = convert_unit(vote)
    info = [title, int(year), float(rating), float(score), vote , director, story, movie_rate, genre1, genre2, lead_actor, sup_actor, image]
    
    return info


In [19]:
info = movie_crawl("헌트", driver)
info


['헌트',
 2022,
 7.2,
 55.0,
 '231',
 'Lee Jung-jae',
 'A spy movie based in the 1980s National Security Planning.',
 '15',
 'Action',
 'Drama',
 'Lee Jung-jae',
 'Jung Woo-sung',
 'https://m.media-amazon.com/images/M/MV5BYzc3MzZmNTEtNzdlMy00NTkyLWIzYmMtNzA1YmY4MTMxOTg3XkEyXkFqcGdeQXVyNjk1NzU1Mjk@._V1_.jpg']

In [20]:
import pickle
with open('model2.pkl', 'rb') as pickle_files:
        model = pickle.load(pickle_files)
column = [
                    'title',
                    'years',
                    'ratings',
                    'scores',
                    'votes',
                    'director',
                    'story',
                    'movie_rate',
                    'genre1',
                    'genre2',
                    'lead_actor',
                    'sup_actor']
movie_info = pd.DataFrame(
                [info[:-1]],
                columns=column)
res = model.predict(movie_info)
result = np.exp(res)

c:\Users\mgd81\anaconda3\envs\s3_project\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- story
Feature names seen at fit time, yet now missing:
- id

  warnings.warn(message, FutureWarning)


ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: 'A spy movie based in the 1980s National Security Planning.'

In [112]:
# title	years	ratings	scores	votes	director	story	movie_rate	genre1	genre2	lead_actor	sup_actor

df = pd.DataFrame([info], columns=['title','years','ratings','scores','votes','director','story','movie_rate','genre1','genre2','lead_actor','sup_actor'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   title       1 non-null      object 
 1   years       1 non-null      int64  
 2   ratings     1 non-null      float64
 3   scores      1 non-null      float64
 4   votes       1 non-null      float64
 5   director    1 non-null      object 
 6   story       1 non-null      object 
 7   movie_rate  1 non-null      object 
 8   genre1      1 non-null      object 
 9   genre2      1 non-null      object 
 10  lead_actor  1 non-null      object 
 11  sup_actor   1 non-null      object 
dtypes: float64(3), int64(1), object(8)
memory usage: 224.0+ bytes


In [2]:
import os
from pathlib import Path

current_path = Path(os.getcwd())
str(current_path)

'c:\\Users\\mgd81\\Section3\\Section3_project'